In [ ]:
# ==================================================
# 1일차 - 2교시: 데이터를 요리하기 위한 재료 손질 (1) & 자동차 데이터 불러오고 살펴보기
# ==================================================
print("\n--- 1일차 2교시 시작 ---")

# 1. Pandas 라이브러리 (이미 불러왔지만, 명시적으로)
import pandas as pd
import numpy as np # '?'를 NaN으로 바꾸기 위해 필요

# 2. 자동차 데이터셋 불러오기
url_autos = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"

# 컬럼 이름 정의 (총 26개)
headers_auto = ["symboling", "normalized-losses", "make", "fuel-type", "aspiration",
                "num-of-doors", "body-style", "drive-wheels", "engine-location",
                "wheel-base", "length", "width", "height", "curb-weight", "engine-type",
                "num-of-cylinders", "engine-size", "fuel-system", "bore", "stroke",
                "compression-ratio", "horsepower", "peak-rpm", "city-mpg", "highway-mpg", "price"]

# CSV 파일 불러오기. '?'를 NaN으로 바로 인식하도록 na_values 옵션 사용
df_cars_original = pd.read_csv(url_autos, header=None, names=headers_auto, na_values="?")
print("자동차 데이터 로드 완료.")

# 원본을 유지하기 위해 복사본으로 작업 (매우 중요!)
df_auto = df_cars_original.copy()
print("작업용 데이터프레임(df_auto) 복사 완료.")

# 3. 불러온 데이터 엿보기
print("\n--- df_auto 맨 앞 5줄 ---")
print(df_auto.head())
print("\n--- df_auto 맨 뒤 3줄 ---")
print(df_auto.tail(3))
print("\n--- df_auto 전체 크기 (행, 열) ---")
print(df_auto.shape)

# 4. 초기 데이터 상태 확인 (info)
print("\n--- 초기 df_auto 종합 정보 (info) ---")
df_auto.info()
# 여기서 'normalized-losses', 'bore', 'stroke', 'horsepower', 'peak-rpm', 'price' 컬럼이
# object 타입이고, Non-Null Count가 전체 행 수보다 적은 것을 확인할 수 있습니다 (NaN 때문에).

print("\n--- 1일차 2교시 실습 종료 ---")


# ==================================================
# 1일차 - 3교시: 데이터를 요리하기 위한 재료 손질 (2) & 시각화 맛보기
# ==================================================
print("\n--- 1일차 3교시 시작 ---")

# 1. 필요한 라이브러리 (Pandas, NumPy는 이전 셀에서 임포트 가정)
import matplotlib.pyplot as plt
%matplotlib inline

# [텍스트 셀 안내]
# (df_auto는 2교시에서 생성 및 초기화되었다고 가정)
# 만약 df_auto가 없다면, 2교시 코드를 먼저 실행해주세요!

# 2. 숫자형/모든 컬럼 기술 통계치 확인 (전처리 전 상태)
print("\n--- 전처리 전 숫자형 컬럼 기술 통계치 ---")
print(df_auto.describe(include=[np.number])) # 숫자형 컬럼만 선택
print("\n--- 전처리 전 모든 컬럼 기술 통계치 ---")
print(df_auto.describe(include='all'))


# ---!!! 'price' 컬럼 전처리 (가장 중요!) !!!---
print("\n--- 'price' 컬럼 전처리 시작 ---")
# na_values="?"로 이미 '?'는 NaN으로 로드되었을 것입니다.
# 여기서는 pd.to_numeric과 fillna에 집중합니다.

# 3-1. 'price' 컬럼을 숫자형(float)으로 변환 (pd.to_numeric 사용)
# errors='coerce'는 숫자로 변환할 수 없는 값을 만나면 NaN으로 만듭니다.
df_auto.loc[:, 'price'] = pd.to_numeric(df_auto['price'], errors='coerce')
print(f"1. pd.to_numeric 후 'price' Dtype: {df_auto['price'].dtype}")

# 3-2. (안전장치) 만약 여전히 object 타입이라면 float으로 강제 변환 시도
if df_auto['price'].dtype == 'object':
    print("   'price'가 object 타입으로 남아있어 astype(float) 시도...")
    try:
        df_auto.loc[:, 'price'] = df_auto['price'].astype(float)
    except ValueError as e:
        print(f"   경고! 'price'를 float으로 변환 중 오류: {e}. 데이터 확인 필요!")
        # 정말 심각한 경우, 숫자가 아닌 값을 모두 NaN으로 바꾸는 apply 로직을 여기에 추가할 수 있습니다.
        # def force_float_or_nan(x):
        #     try: return float(x)
        #     except: return np.nan
        # df_auto.loc[:, 'price'] = df_auto['price'].apply(force_float_or_nan).astype(float)
print(f"2. 최종 형변환 시도 후 'price' Dtype: {df_auto['price'].dtype}")


# 3-3. 'price' 컬럼의 NaN 값을 평균으로 채우기
# 먼저 평균을 계산합니다. 이 때 'price'는 이미 숫자형(또는 모든 값이 NaN인 숫자형)이어야 합니다.
avg_price = df_auto['price'].mean() # .mean()은 NaN을 알아서 제외하고 계산

if pd.isna(avg_price): # 만약 'price' 컬럼의 모든 유효한 값이 NaN이어서 평균 계산이 불가능했다면
    print("경고: 'price' 컬럼의 평균을 계산할 수 없습니다 (모든 값이 NaN일 수 있음). NaN을 0으로 채웁니다.")
    df_auto.loc[:, 'price'] = df_auto['price'].fillna(0)
else:
    df_auto.loc[:, 'price'] = df_auto['price'].fillna(avg_price)
    print(f"3. 'price' 컬럼의 NaN 값을 평균값 ({avg_price:.2f})으로 채우기 완료.")

print("\n--- 'price' 컬럼 전처리 후 종합 정보 (df_auto.info()) ---")
df_auto.info() # 'price' 컬럼이 float64이고, Non-Null Count가 증가했는지 확인!
# ---!!! 'price' 컬럼 전처리 끝 !!!---


# 4. 자동차 가격 분포 히스토그램 그려보기
print("\n--- 자동차 가격 분포 히스토그램 ---")

# 히스토그램 그리기 전, 'price' 컬럼이 확실히 숫자형인지 최종 확인
if pd.api.types.is_numeric_dtype(df_auto['price']):
    price_data_for_hist = df_auto['price'].dropna() # 혹시 모를 NaN 최종 제거
    if not price_data_for_hist.empty:
        plt.figure(figsize=(10, 6))
        plt.hist(price_data_for_hist, bins=25, edgecolor='black', color='deepskyblue') # bins 값 조정 가능
        plt.title('Distribution of Car Prices', fontsize=16)
        plt.xlabel('Price (USD)', fontsize=13)
        plt.ylabel('Frequency (Number of Cars)', fontsize=13)
        plt.grid(axis='y', linestyle='--', alpha=0.7) # y축 격자만 표시
        plt.show()
    else:
        print("경고: 'price' 컬럼에 유효한 숫자 데이터가 없어 히스토그램을 그릴 수 없습니다.")
else:
    print("치명적 경고: 'price' 컬럼이 최종적으로 숫자형이 아닙니다. 히스토그램을 그릴 수 없습니다.")
    print(f"  현재 'price' 컬럼 Dtype: {df_auto['price'].dtype}")
    print("  'price' 컬럼 샘플:\n", df_auto['price'].head(10))

print("\n--- 1일차 3교시 실습 종료 ---")